# demo train with `wandb`

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import os, wandb, time, random
from wandb.keras import WandbCallback
from resnet34 import ResNet34

In [2]:
# load data
from tensorflow.keras.datasets import cifar10
import keras

(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
num_classes = 10
input_shape = X_train[0].shape
print(input_shape)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
Y_train = keras.utils.np_utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.np_utils.to_categorical(Y_test, num_classes)

(32, 32, 3)


In [3]:
# wandb run config
wandb.login()
config = dict(epochs=25,
              batch_size=128,
              latent=1024,
              loss_fn='categorical_crossentropy',
              optimizer='adam',
              architecture='resnet34',
              activation='softmax')
run = wandb.init(project="cifar-10-grid", 
                config=config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: h3x4g0n. Use `wandb login --relogin` to force relogin
/home/hexagon/miniconda3/envs/tf37/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
# model init
model = ResNet34(latent=1024, n_classes=num_classes, activation=config["activation"])
model.build(input_shape=(1, *X_train[0].shape))
print(model.summary(X_train[0].shape))
model.compile(optimizer=config["optimizer"], loss=config["loss_fn"], metrics=['accuracy'])

2022-06-16 23:36:51.251013: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 23:36:51.278132: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 23:36:51.278288: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 23:36:51.278667: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 64)        9472      
                                                                 
 batch_normalization (BatchN  (None, 16, 16, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 16, 16, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 8, 64)         0         
 )                                                               
                                                                 
 residual_block (ResidualBlo  (None, 8, 8, 64)         74368 

In [5]:
# model train with WandbCallback
val_images, val_labels = X_test[:32], Y_test[:32]

model.fit(X_train, Y_train,
            epochs=config["epochs"], 
            batch_size=config["batch_size"],
            validation_data=(X_test, Y_test),
            callbacks=[WandbCallback()])

run.finish()

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/25


2022-06-16 23:36:55.823570: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8400
2022-06-16 23:36:56.743380: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


391/391 [==============================] - ETA: 0s - loss: 1.6330 - accuracy: 0.4440

wandb: ERROR Can't save model in the h5py format. The model will be saved as W&B Artifacts in the SavedModel format.
2022-06-16 23:37:09.954279: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


...

391/391 [==============================] - 17s 43ms/step - loss: 0.8025 - accuracy: 0.7186 - val_loss: 1.0738 - val_accuracy: 0.6240 - _timestamp: 1655447869.0000 - _runtime: 64.0000
Epoch 5/25
391/391 [==============================] - 9s 23ms/step - loss: 0.7022 - accuracy: 0.7553 - val_loss: 1.3105 - val_accuracy: 0.5879 - _timestamp: 1655447886.0000 - _runtime: 81.0000
Epoch 6/25
390/391 [============================>.] - ETA: 0s - loss: 0.6045 - accuracy: 0.7876

INFO:tensorflow:Assets written to: /home/hexagon/Code/m1-ml/models/wandb/run-20220616_233645-y285mkde/files/model-best/assets


INFO:tensorflow:Assets written to: /home/hexagon/Code/m1-ml/models/wandb/run-20220616_233645-y285mkde/files/model-best/assets
wandb: Adding directory to artifact (/home/hexagon/Code/m1-ml/models/wandb/run-20220616_233645-y285mkde/files/model-best)... Done. 0.6s


391/391 [==============================] - 17s 44ms/step - loss: 0.6047 - accuracy: 0.7877 - val_loss: 0.9136 - val_accuracy: 0.6912 - _timestamp: 1655447895.0000 - _runtime: 90.0000
Epoch 7/25
391/391 [==============================] - 9s 23ms/step - loss: 0.5346 - accuracy: 0.8135 - val_loss: 1.0274 - val_accuracy: 0.6657 - _timestamp: 1655447913.0000 - _runtime: 108.0000
Epoch 8/25
391/391 [==============================] - 9s 23ms/step - loss: 0.4592 - accuracy: 0.8396 - val_loss: 0.9228 - val_accuracy: 0.7032 - _timestamp: 1655447922.0000 - _runtime: 117.0000
Epoch 9/25
391/391 [==============================] - 9s 22ms/step - loss: 0.4222 - accuracy: 0.8539 - val_loss: 1.2284 - val_accuracy: 0.6353 - _timestamp: 1655447930.0000 - _runtime: 125.0000
Epoch 10/25
391/391 [==============================] - 9s 23ms/step - loss: 0.3394 - accuracy: 0.8819 - val_loss: 0.9430 - val_accuracy: 0.7223 - _timestamp: 1655447939.0000 - _runtime: 134.0000
Epoch 11/25
391/391 [===================

accuracy,▁▃▄▅▅▆▆▆▆▇▇▇▇████████████
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▆▅▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▄▅▄▇▆▇▆█▇▆▇▇███▆▇████▇█
val_loss,█▁▁▁▂▁▁▁▂▁▁▂▁▂▁▁▂▂▂▂▂▂▂▂▂
accuracy,0.97532
best_epoch,5
best_val_loss,0.91357
epoch,24
loss,0.07124
val_accuracy,0.7314
